# VAE Model Testing Framework

## 1. Imports

In [1]:
import torch
import numpy as np
import torch.utils
import matplotlib.pyplot as plt
import torch.optim as optim

import utils.visualize
from trainers import UnsupervisedTrainer
import losses
import models
from datasets import get_dataset
from utils.io import find_optimal_num_workers

## 2. Configuration

In [2]:
# --- General Hyperparameters ---
model_name = 'vae_burgess'  # Name of the model architecture file (e.g., 'vae_burgess')
latent_dim = 10
learning_rate = 1e-4
num_epochs = 20
batch_size = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
rec_dist = 'bernoulli'  # Reconstruction distribution (e.g., 'bernoulli', 'gaussian')

In [3]:
# --- Loss Specific Hyperparameters ---
# Beta VAE
loss_name = 'betavae'      # Type of loss ('betavae', 'annealedvae', 'betatcvae')
loss_kwargs = {'beta': 4.0,
               'rec_dist': rec_dist}

In [4]:
# AnnealedVAE
loss_name = 'annealedvae'  
loss_kwargs = {
    'C_init': 0.0,
    'C_fin_3dshapes': 25.0,
    'C_fin_dsprites': 15.0,
    'gamma_annealed': 100.0,
    'anneal_steps': 10000,
    'rec_dist': rec_dist,
}

In [5]:
# BetaTCVAE
loss_name = 'betatcvae'
loss_kwargs = {
    'alpha_tc': 1.0,
    'beta_tc_3dshapes': 6.0,
    'beta_tc_dsprites': 4.0,
    'gamma_tc': 1.0,
    'is_mss': True,
    'rec_dist': rec_dist
}

In [6]:
# Factor VAE

loss_name = 'factorvae'
loss_kwargs = {
    'device': device,
    'gamma': 6.4,
    'discr_lr': 5e-5,
    'discr_betas': (0.5, 0.9),
    'rec_dist': rec_dist,
}

## 3. Load Datasets

In [7]:
# Load 3D Shapes
Shapes3D = get_dataset("shapes3d")
shapes3d_dataset = Shapes3D(selected_factors='all', not_selected_factors_index_value=None)
num_workers_3dshapes = find_optimal_num_workers(shapes3d_dataset, batch_size=batch_size)
shapes3d_dataloader = torch.utils.data.DataLoader(shapes3d_dataset, batch_size=batch_size, num_workers=num_workers_3dshapes, shuffle=True, pin_memory=True)
print(f"Loaded 3D Shapes dataset with {len(shapes3d_dataset)} samples.")

# Load dSprites
Dsprites = get_dataset('dsprites')
dsprites_dataset = Dsprites(selected_factors='all', not_selected_factors_index_value=None)
num_workers_dsprites = find_optimal_num_workers(dsprites_dataset, batch_size=batch_size)
dsprites_dataloader = torch.utils.data.DataLoader(dsprites_dataset, batch_size=batch_size, num_workers=num_workers_dsprites, shuffle=True, pin_memory=True)
print(f"Loaded dSprites dataset with {len(dsprites_dataset)} samples.")

Defaulting max_workers to physical core count: 8
Finding optimal num_workers (testing 0 to 8)...
  num_workers=0: Warming up with 5 batches...
  num_workers=0: 0.00917 sec/batch
  num_workers=1: Warming up with 5 batches...
  num_workers=1: 0.00529 sec/batch
  num_workers=2: Warming up with 5 batches...
  num_workers=2: 0.00278 sec/batch
  num_workers=3: Warming up with 5 batches...
  num_workers=3: 0.00262 sec/batch
  num_workers=4: Warming up with 5 batches...
  num_workers=4: 0.00278 sec/batch
  num_workers=5: Warming up with 5 batches...
  num_workers=5: 0.00215 sec/batch
  num_workers=6: Warming up with 5 batches...
  num_workers=6: 0.00426 sec/batch
  num_workers=7: Warming up with 5 batches...
  num_workers=7: 0.00269 sec/batch
  num_workers=8: Warming up with 5 batches...
  num_workers=8: 0.00351 sec/batch
Optimal num_workers: 5 (0.00215 sec/batch)
Loaded 3D Shapes dataset with 480000 samples.
Defaulting max_workers to physical core count: 8
Finding optimal num_workers (testing

## 4. Setup Model, Loss, and Optimizer

In [ ]:
def setup_components(dataset):
    """Instantiates model, loss function, and optimizer based on config."""
    img_size = dataset[0][0].shape
    n_data = len(dataset)
    

    # Instantiate Model
    model = models.select(model_name, img_size, latent_dim)

    if loss_name == 'betatcvae':
        loss_kwargs['n_data'] = n_data
    
    loss_fn = losses.select(loss_name, **loss_kwargs)

    # Instantiate Optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"--- Setup for {dataset.__class__.__name__} --- ")
    print(f"Model: {model.__class__.__name__}")
    print(f"Loss: {loss_fn.__class__.__name__} (rec_dist={rec_dist}), kwargs={loss_kwargs}")
    print(f"Optimizer: {optimizer.__class__.__name__}")
    print(f"---------------------------")

    return model, loss_fn, optimizer

## 5. Train and Visualize

## 5.1 - 3D Shapes

In [ ]:
print("\n===== Training on 3D Shapes =====")
model_3dshapes, loss_fn_3dshapes, optimizer_3dshapes = setup_components(shapes3d_dataset, 'shapes3d')

trainer_3dshapes = UnsupervisedTrainer(model=model_3dshapes,
                                     loss_fn=loss_fn_3dshapes,
                                     scheduler=None,
                                     optimizer=optimizer_3dshapes,
                                     device=device)

trainer_3dshapes.train(shapes3d_dataloader, num_epochs=num_epochs)

In [ ]:
print("\n===== Visualizing 3D Shapes Results =====")
visualizer_3dshapes = utils.visualize.Visualizer(vae_model=model_3dshapes, dataset=shapes3d_dataset)

print("Plotting random reconstructions...")
visualizer_3dshapes.plot_random_reconstructions(10, mode='mean')
plt.show()

print("Plotting reconstructions from specific indices...")
indices_3dshapes = [5000, 6000, 7000, 100, 1000, 1024] # Example indices
visualizer_3dshapes.plot_reconstructions_sub_dataset(indices_3dshapes, mode='mean')
plt.show()

print("Plotting latent traversals...")
visualizer_3dshapes.plot_all_latent_traversals(num_samples=20)
plt.show()

## 5.2. Train and Visualize: dSprites

In [ ]:
print("\n===== Training on dSprites =====")
model_dsprites, loss_fn_dsprites, optimizer_dsprites = setup_components(dsprites_dataset, 'dsprites')

trainer_dsprites = UnsupervisedTrainer(model=model_dsprites,
                                     loss_fn=loss_fn_dsprites,
                                     scheduler=None,
                                     optimizer=optimizer_dsprites,
                                     device=device)

trainer_dsprites.train(dsprites_dataloader, num_epochs=num_epochs)

In [ ]:
print("\n===== Visualizing dSprites Results =====")
visualizer_dsprites = utils.visualize.Visualizer(vae_model=model_dsprites, dataset=dsprites_dataset)

print("Plotting random reconstructions...")
visualizer_dsprites.plot_random_reconstructions(10, mode='mean')
plt.show()

print("Plotting reconstructions from specific indices...")
indices_dsprites = [0, 10000, 20000, 30000, 40000, 50000] # Example indices
visualizer_dsprites.plot_reconstructions_sub_dataset(indices_dsprites, mode='mean')
plt.show()

print("Plotting latent traversals...")
visualizer_dsprites.plot_all_latent_traversals(num_samples=20)
plt.show()